# 模型可解释性分析 - Alpha-Hunter

本notebook用于：
1. 分析Transformer的注意力权重（如果可用）
2. 分析PCA因子的重要性
3. 特征贡献分析
4. 预测vs实际收益的关系
5. 时间序列模式分析


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.data_loader import SequenceDataLoader
from src.evaluator import PerformanceEvaluator

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 20)
print("Libraries loaded!")


## 1. 加载数据和结果


In [ ]:
# 加载PCA特征
data_loader = SequenceDataLoader(pca_path='../feature/pca_feature_store.csv', sequence_length=12)
pca_df = data_loader.df

print(f"PCA Features: {data_loader.feature_columns}")
print(f"Total records: {len(pca_df)}")
pca_df.head()


In [ ]:
# 加载预测结果（选择最佳模型）
import glob
results_dir = Path('../results')

# 尝试加载transformer预测（通常性能较好）
model_name = 'transformer'
pred_files = glob.glob(str(results_dir / model_name / 'predictions_*.csv'))

if pred_files:
    latest = max(pred_files, key=lambda x: Path(x).stat().st_mtime)
    predictions = pd.read_csv(latest)
    predictions['date'] = pd.to_datetime(predictions['date'])
    print(f"Loaded predictions: {len(predictions)} records")
    print(f"Date range: {predictions['date'].min().date()} to {predictions['date'].max().date()}")
else:
    print("⚠️ No predictions found. Please run training first.")
    predictions = None


## 2. PCA因子分析

分析PCA因子的时间序列特征和分布


In [ ]:
# PCA因子的时间序列均值
pca_features = [col for col in pca_df.columns if col.startswith('pca_')]
pca_time_series = pca_df.groupby('date')[pca_features].mean()

fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.flatten()

for i, col in enumerate(pca_features):
    if i < len(axes):
        axes[i].plot(pca_time_series.index, pca_time_series[col], linewidth=1.5, color='steelblue')
        axes[i].set_title(f'{col.upper()} 时间序列')
        axes[i].set_ylabel('均值')
        axes[i].grid(True, alpha=0.3)
        axes[i].tick_params(axis='x', rotation=45)

# 隐藏多余的subplot
for i in range(len(pca_features), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(results_dir / 'pca_time_series.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# PCA因子相关性矩阵
# 随机抽样以加快计算
sample_dates = np.random.choice(pca_df['date'].unique(), size=min(50, len(pca_df['date'].unique())), replace=False)
sample_df = pca_df[pca_df['date'].isin(sample_dates)]

pca_corr = sample_df[pca_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(pca_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title('PCA因子相关性矩阵')
plt.tight_layout()
plt.savefig(results_dir / 'pca_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("PCA因子应该是正交的（相关性接近0）")


## 3. 预测质量分析

分析预测的准确性和一致性


In [ ]:
if predictions is not None:
    # 预测vs实际散点图（分时间段）
    predictions['year'] = predictions['date'].dt.year
    years = predictions['year'].unique()[-3:]  # 最近3年
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i, year in enumerate(years):
        if i < 3:
            year_data = predictions[predictions['year'] == year]
            sample = year_data.sample(min(2000, len(year_data)))
            
            axes[i].scatter(sample['prediction'], sample['actual_return'], 
                          alpha=0.3, s=10, color='steelblue')
            axes[i].set_title(f'{year}年 预测 vs 实际')
            axes[i].set_xlabel('预测收益')
            axes[i].set_ylabel('实际收益')
            axes[i].axhline(y=0, color='red', linestyle='--', alpha=0.3)
            axes[i].axvline(x=0, color='red', linestyle='--', alpha=0.3)
            axes[i].grid(True, alpha=0.3)
            
            # 添加相关系数
            corr = sample[['prediction', 'actual_return']].corr().iloc[0, 1]
            axes[i].text(0.05, 0.95, f'Corr: {corr:.3f}', 
                        transform=axes[i].transAxes, 
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(results_dir / 'prediction_vs_actual.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
if predictions is not None:
    # IC随时间的稳定性
    evaluator = PerformanceEvaluator()
    ic_series = evaluator.compute_ic(predictions)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # IC时间序列
    axes[0].plot(ic_series.index, ic_series.values, linewidth=1.5, alpha=0.7)
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[0].axhline(y=ic_series.mean(), color='green', linestyle='--', alpha=0.5, 
                    label=f'Mean: {ic_series.mean():.3f}')
    axes[0].fill_between(ic_series.index, 0, ic_series.values, alpha=0.2)
    axes[0].set_title('IC时间序列')
    axes[0].set_ylabel('IC')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # IC滚动均值
    rolling_ic = ic_series.rolling(window=12).mean()
    axes[1].plot(rolling_ic.index, rolling_ic.values, linewidth=2, color='darkblue', label='12月滚动均值')
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[1].set_title('IC 12月滚动均值')
    axes[1].set_ylabel('IC')
    axes[1].set_xlabel('日期')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'ic_stability.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"IC均值: {ic_series.mean():.4f}")
    print(f"IC标准差: {ic_series.std():.4f}")
    print(f"IC正值比例: {(ic_series > 0).mean():.2%}")


## 4. 资产特征分析

分析哪些资产更容易被准确预测


In [ ]:
if predictions is not None:
    # 计算每个资产的平均预测误差
    from scipy import stats
    
    asset_stats = predictions.groupby('asset').apply(
        lambda x: pd.Series({
            'count': len(x),
            'mean_pred': x['prediction'].mean(),
            'mean_actual': x['actual_return'].mean(),
            'correlation': stats.spearmanr(x['prediction'], x['actual_return'])[0] if len(x) > 10 else np.nan,
            'mae': np.abs(x['prediction'] - x['actual_return']).mean(),
        })
    )
    
    # 过滤至少有20个预测的资产
    asset_stats = asset_stats[asset_stats['count'] >= 20].sort_values('correlation', ascending=False)
    
    print(f"\\n分析的资产数量: {len(asset_stats)}")
    print(f"\\n预测相关性最高的10个资产:")
    print(asset_stats.head(10)[['count', 'correlation', 'mae']].round(3))
    
    print(f"\\n预测相关性最低的10个资产:")
    print(asset_stats.tail(10)[['count', 'correlation', 'mae']].round(3))


In [ ]:
if predictions is not None:
    # 可视化资产相关性分布
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 相关性分布
    axes[0].hist(asset_stats['correlation'].dropna(), bins=30, alpha=0.7, 
                edgecolor='black', color='steelblue')
    axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
    axes[0].axvline(x=asset_stats['correlation'].median(), color='green', 
                   linestyle='--', alpha=0.5, label=f"Median: {asset_stats['correlation'].median():.3f}")
    axes[0].set_title('资产级别预测相关性分布')
    axes[0].set_xlabel('Spearman相关系数')
    axes[0].set_ylabel('资产数量')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 预测次数 vs 相关性
    axes[1].scatter(asset_stats['count'], asset_stats['correlation'], alpha=0.5, s=20)
    axes[1].set_title('预测次数 vs 相关性')
    axes[1].set_xlabel('预测次数')
    axes[1].set_ylabel('相关系数')
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.3)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'asset_level_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()


## 5. PCA因子对预测的贡献

分析哪些PCA因子对预测最重要（需要原始PCA数据）


In [ ]:
# 读取PCA解释方差（如果存在）
var_file = Path('../feature/pca_explained_variance.csv')
if var_file.exists():
    var_df = pd.read_csv(var_file)
    var_df['date'] = pd.to_datetime(var_df['date'])
    
    # 可视化解释方差
    pca_cols = [col for col in var_df.columns if col.startswith('pca_')]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 平均解释方差
    mean_var = var_df[pca_cols].mean()
    axes[0].bar(range(len(mean_var)), mean_var.values, alpha=0.7, color='steelblue')
    axes[0].set_title('各PCA因子的平均解释方差')
    axes[0].set_xlabel('PCA因子')
    axes[0].set_ylabel('解释方差比例')
    axes[0].set_xticks(range(len(mean_var)))
    axes[0].set_xticklabels([f'PC{i+1}' for i in range(len(mean_var))], rotation=45)
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 累计解释方差
    cumsum_var = mean_var.cumsum()
    axes[1].plot(range(len(cumsum_var)), cumsum_var.values, marker='o', linewidth=2, markersize=8)
    axes[1].axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='80%阈值')
    axes[1].set_title('累计解释方差')
    axes[1].set_xlabel('PCA因子数量')
    axes[1].set_ylabel('累计解释方差')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'pca_explained_variance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\\n前5个因子的累计解释方差: {cumsum_var.iloc[4]:.2%}")
else:
    print("未找到PCA解释方差文件")


## 总结

本notebook完成了：
1. ✅ PCA因子的时间序列和相关性分析
2. ✅ 预测质量和IC稳定性分析  
3. ✅ 资产级别的预测准确性分析
4. ✅ PCA因子的解释方差分析

### 关键发现

查看上面的输出和图表，重点关注：
- **IC稳定性**：IC是否在大部分时间为正
- **PCA因子贡献**：前几个因子解释了多少方差
- **资产差异**：哪些资产更容易/难以预测

### 改进建议

基于分析结果，可以考虑：
1. 调整PCA因子数量（增加或减少）
2. 对难以预测的资产进行特殊处理
3. 在IC较低的时期调整模型权重
4. 探索不同的超参数组合

所有分析图表已保存到 `results/` 目录。
